# Preparing Dolphin

This notebook filters Dolphin for single variable equations and gives a clean text

In [1]:
import json, csv
import numpy as np
import pandas as pd
import re
import warnings

warnings.filterwarnings('ignore')

print("Libraries Imported")

Libraries Imported


In [2]:
def getText(doc):
  doc =  str(doc)
  doc = doc.lower().strip()
  doc = re.sub('\n', ' ', doc)
  doc = re.sub(r'\s+', ' ', doc)
  m = re.search(r'<meta property=\"og:title\" content=\"(.*?)\"/>',doc)
  m1 = re.search(r'<meta property=\"og:description\" content=\"(.*?)\"/>',doc)

  if m != None and m1!= None:
    text = str(m.group(1)) + ' ' + str(m1.group(1))
  else:
    text  = "No match"

  return text

Preparing Datasets
--

In [3]:
data = pd.read_json('dev_cleaned.json')

for i, row in data.iterrows():
  if re.match(r"^\"/><meta property=\"", row['original_text']) == None:
    text = getText(row['original_text'])
    if text != "No match":
      data.at[i,'text'] = text
  items = row['equations'].split("\r\n")
  data.at[i,'nequ'] = len(items)

In [8]:
data.head()

,original_text,text,sources,flag,ans,equations,id,nequ
0,"""/><meta property=""og:site_name"" content=""Yaho...",One number is 3 less than a second number. Twi...,[https://answers.yahoo.com/question/index?qid=...,0,6; 9,"unkn: x,y\r\nequ: x + 3 = y\r\nequ: 2*y + 12 =...",yahoo.answers.20091227181039aazhen8,3.0
1,"""/><meta property=""og:site_name"" content=""Yaho...",A number is 12 less than another. The sum of ...,[https://answers.yahoo.com/question/index?qid=...,0,{8; 20},"unkn: m,n\r\nequ: m = n - 12\r\nequ: m + n = 28",yahoo.answers.20070524141453aahh0nv,3.0
2,"""/><meta property=""og:site_name"" content=""Yaho...","The sum of two numbers is twenty-three, and th...",[https://answers.yahoo.com/question/index?qid=...,0,{9; 14},"unkn: x,y\r\nequ: x + y = 23\r\nequ: x = 5 + y",yahoo.answers.20091207064212aaklfpn,3.0
3,"""/><meta property=""og:site_name"" content=""Yaho...","Find two numbers with sum 10 and product 20.""/...",[https://answers.yahoo.com/question/index?qid=...,0,{2.764; 7.236},"unkn: x, y\r\nequ: x + y = 10\r\nequ: x * y = 20",yahoo.answers.20070527111158aax4f93,3.0
4,"""/><meta property=""og:site_name"" content=""Yaho...",The sum of two numbers is 91. The larger numb...,[https://answers.yahoo.com/question/index?qid=...,0,{18; 73},"unkn: x,y\r\nequ: x + y = 91\r\nequ: x = 1 + 4*y",yahoo.answers.20080911174339aamgdzd,3.0


In [9]:
data = data.loc[data['nequ'] == 2]

data[["unknowns","equations"]] = data["equations"].str.split("\r\n", 1, expand = True)

data["unknowns"] = data["unknowns"].str[6:]

data["equations"] = data["equations"].str[5:]

data = data[["text","ans","equations","unknowns"]]

In [14]:
print(data.shape)
data.head()

(1571, 4)


,text,ans,equations,unknowns
5,What is the sum of the distinct prime factors ...,226,s = 3 + 223,s
6,Fifteen more than four times a number is 6 mor...,9,15 + 4*n = 6 + 5*n,n
7,5 less than 6 times a whole number is 9 less t...,2,6*n - 5 = 8*n - 9,n
10,1/3 of a number is 5 less than half of the sam...,30,1/3 * x = 1/2 * x - 5,x
11,I think of a number multiply it by 8 then subt...,11,8*x - 66 = 2*x,x


In [15]:
data = data.dropna(subset=["equations"])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1571 entries, 5 to 3388
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       1571 non-null   object
 1   ans        1571 non-null   object
 2   equations  1571 non-null   object
 3   unknowns   1571 non-null   object
dtypes: object(4)
memory usage: 61.4+ KB


Data Cleaning
--

In [19]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

# import spacy

# import string

import re

# nlp = spacy.load("en")

nltk_stopwords = set(stopwords.words('english'))

# spacy_stopwords = nlp.Defaults.stop_words

stopset = nltk_stopwords#.union(spacy_stopwords)

stopset.difference_update(["a","more","less","than","one","two","three","four","five","six","seven","eight","nine","ten","eleven","twelve","fifteen","twenty","forty","sixty","fifty","hundred","once","first","second","third"])

punctuation = "!\"#$&',;?@\_`{|}~"

def getText(doc):
  doc =  str(doc)
  doc = doc.lower().strip()
  doc = re.sub('\n', ' ', doc)
  doc = re.sub(r'\s+', ' ', doc)
  m = re.search(r'<meta property=\"og:title\" content=\"(.*?)\"/>',doc)
  m1 = re.search(r'<meta property=\"og:description\" content=\"(.*?)\"/>',doc)

  if m != None and m1!= None:
    text = str(m.group(1)) + ' ' + str(m1.group(1))
  else:
    text  = "No match"

  return text


def cleanData(doc):
  doc = str(doc)
  doc = doc.lower().strip()
  doc = re.sub('\n', ' ', doc)
  doc = re.sub(r'\s+', ' ', doc)
  pattern = '"/><meta '
  lookup = re.search(pattern, doc)
  if lookup != None:
    index = doc.find(lookup.group(0))
    doc = doc[:index]
  doc = doc.replace('yahoo answers','')
  regex1 = r"[A-Za-z\s*]+[\.:%-][A-Za-z\s*]*"
  regex2 = r"[0-9]+\.[0-9]+"
  regex3 = r"[0-9\s*]{1,}[\=\*+-][0-9\s*]{1,}"
  match1 = re.search(regex1, doc)
  match2 = re.search(regex2, doc)
  match3 = re.search(regex3, doc)
  # tokens = nltk.WordPunctTokenizer().tokenize(doc)      
  tokens = doc.split()
  # clean = [token.lower().strip() for token in tokens if token not in stopset]
  clean = [token for token in tokens if token not in stopset]
  clean = [token.replace('.','') for token in clean if not re.search(r"[0-9]+\.[0-9]+", token)]
  clean = " ".join(clean)
  # clean = [''.join(c for c in s if c not in punctuation) for s in clean]
  final = ''.join(c for c in clean if c not in punctuation)
  # final_tokens = final.split()
  # clean2 = [token.replace('.','') for token in final_tokens if not re.search(r"[0-9]+\.[0-9]+", token)]
  # final = " ".join(clean2)
  # final = " ".join(clean)
  try:
      final = remove_whitespace(final.encode('latin1').decode('utf-8','replace').encode('ascii','ignore'))
  except:
      final = remove_whitespace(final)
  final=final.strip()
  return final

def remove_whitespace(x):
  """
  Helper function to remove any blank space from a string
  x: a string
  """
  try:
      # Remove spaces inside of the string
      x = " ".join(x.split())
  except:
      pass
  return x
  
print("Functions Defined!")


Functions Defined!


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shrij\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
data['cleaned_text'] = data.apply(lambda x: cleanData(str(x['text'])), axis=1)

data['cleaned_text'] = data['cleaned_text'].str.decode("utf-8")

data = data.dropna(subset=["cleaned_text"])

data.info()

data.to_csv("trainData_univariable.csv", index = False)
# data.to_csv("trainData_univariable.txt", index = False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1536 entries, 5 to 3388
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text          1536 non-null   object
 1   ans           1536 non-null   object
 2   equations     1536 non-null   object
 3   unknowns      1536 non-null   object
 4   cleaned_text  1536 non-null   object
dtypes: object(5)
memory usage: 72.0+ KB


In [40]:
pd.options.display.max_colwidth = 1000
data.head()

,text,ans,equations,unknowns,cleaned_text
5,"What is the sum of the distinct prime factors of 2007?""/><meta property=""og:site_name"" content=""Yahoo Answers""/><meta property=""og:image"" content=""https://s.yimg.com/cv/apiv2/social/images/yahoo_default_logo.png""/><meta property=""og:image:width"" content=""500""/><meta property=""og:image:height"" content=""500""/><meta property=""og:title"" content=""Yahoo Answers""/><meta property=""og:type"" content=""website""/><meta property=""og:locale"" content=""zh_TW""/><meta property=""og:url"" content=""https://answers.yahoo.com""/><meta property=""og:description"" content=""""/><meta name=""oath:guce:consent-host"" content=""guce.yahoo.com""/><meta name=""format-detection"" content=""telephone=no""/><link rel=""apple-touch-icon-precomposed"" href=""https://s.yimg.com/tr/54d7a352d4c11dfeb7bd74c82f3fbcff.png""/><link rel=""stylesheet"" type=""text/css"" href=""https://s.yimg.com/zq/ans/frontend/vendor.fc9104bfe51e6f7bf7f6.css""/><link rel=""stylesheet"" type=""text/css"" href=""https://s.yimg.com/zq/ans/frontend/errorPage.4ba4784a22d4cca...",226,s = 3 + 223,s,sum distinct prime factors 2007
6,"Fifteen more than four times a number is 6 more than five times the number. what's the value of the number?""/><meta property=""og:site_name"" content=""Yahoo Answers""/><meta property=""og:image"" content=""https://s.yimg.com/cv/apiv2/social/images/yahoo_default_logo.png""/><meta property=""og:image:width"" content=""500""/><meta property=""og:image:height"" content=""500""/><meta property=""og:title"" content=""Yahoo Answers""/><meta property=""og:type"" content=""website""/><meta property=""og:locale"" content=""zh_TW""/><meta property=""og:url"" content=""https://answers.yahoo.com""/><meta property=""og:description"" content=""""/><meta name=""oath:guce:consent-host"" content=""guce.yahoo.com""/><meta name=""format-detection"" content=""telephone=no""/><link rel=""apple-touch-icon-precomposed"" href=""https://s.yimg.com/tr/54d7a352d4c11dfeb7bd74c82f3fbcff.png""/><link rel=""stylesheet"" type=""text/css"" href=""https://s.yimg.com/zq/ans/frontend/vendor.fc9104bfe51e6f7bf7f6.css""/><link rel=""stylesheet"" type=""text/css"" href=""https:/...",9,15 + 4*n = 6 + 5*n,n,fifteen more than four times a number 6 more than five times number whats value number
7,"5 less than 6 times a whole number is 9 less than 8 times that number. Find the number.""/><meta property=""og:site_name"" content=""Yahoo Answers""/><meta property=""og:image"" content=""https://s.yimg.com/cv/apiv2/social/images/yahoo_default_logo.png""/><meta property=""og:image:width"" content=""500""/><meta property=""og:image:height"" content=""500""/><meta property=""og:title"" content=""Yahoo Answers""/><meta property=""og:type"" content=""website""/><meta property=""og:locale"" content=""zh_TW""/><meta property=""og:url"" content=""https://answers.yahoo.com""/><meta property=""og:description"" content=""""/><meta name=""oath:guce:consent-host"" content=""guce.yahoo.com""/><meta name=""format-detection"" content=""telephone=no""/><link rel=""apple-touch-icon-precomposed"" href=""https://s.yimg.com/tr/54d7a352d4c11dfeb7bd74c82f3fbcff.png""/><link rel=""stylesheet"" type=""text/css"" href=""https://s.yimg.com/zq/ans/frontend/vendor.fc9104bfe51e6f7bf7f6.css""/><link rel=""stylesheet"" type=""text/css"" href=""https://s.yimg.com/zq/ans/f...",2,6*n - 5 = 8*n - 9,n,5 less than 6 times a whole number 9 less than 8 times number find number
10,"1/3 of a number is 5 less than half of the same number. what is the number?""/><meta property=""og:site_name"" content=""Yahoo Answers""/><meta property=""og:image"" content=""https://s.yimg.com/cv/apiv2/social/images/yahoo_default_logo.png""/><meta property=""og:image:width"" content=""500""/><meta property=""og:image:height"" content=""500""/><meta property=""og:title"" content=""Yahoo Answers""/><meta property=""og:type"" content=""website""/><meta property=""og:locale"" content=""zh_TW""/><meta pr